# Cube in a Box

### Visualizing Data


The [OpenDatacube](https://github.com/opendatacube) provides an integrated gridded data analysis environment capable of delivering decades of analysis ready earth observation satellite and related data from multiple satellite and other acquisition systems.

Cube in a box is an easy to install implementation of the OpenDataCube that provides small case study notebook examples of the open datacubes capabilities.



Cube in a box automatically indexes the landsat path/rows stored in Amazon's [Landsat archive](https://registry.opendata.aws/landsat-8)  associated with the extent specified in the 'cube in a box' config file. This notebook demonstrates how to load a specific time slice and visualise sensor bands and also how to do a pixel drill.

This notebook touches briefly on some the implimented features of the Datacube module, and is only intended to demonstrate functionality.

In [ ]:
%matplotlib inline
import datacube #Load the datacube library
import folium
import datetime
import json
from utils.utils import threeBandImage
from utils.utils import loadConfigExtent
from utils.utils import transformToWGS


Let's first explore the extent area that had been specified in the config file. 
Zoom in to the red bounding box in the below map to confirm this is the extent that you're looking to experiment explore. If the config parameters have been set correctly there will be a bounding box over your case study area, if the bounding box is in the right location then Cube in a Box has indexed all relavent Landsat 8 path/rows corresponding the extent.

In [ ]:
# Load the extents that were saved after the initial index
config = json.load(open('/opt/odc/data/configIndex.txt'))
lon_min, lon_max, lat_min, lat_max = config['extent']

centre = [(lon_min + lon_max)/2, (lat_min + lat_max)/2]
rectangle =  [[lat_max, lon_min], [lat_max, lon_max], [lat_min, lon_max] ,[lat_min, lon_min], [lat_max, lon_min]]

# Create a map
m = folium.Map(location=centre, zoom_start=10)
bounds = folium.PolyLine(rectangle, color='red')

# Zoom to our area of interest
m.fit_bounds([[lat_min, lon_min],[lat_max, lon_max]])
bounds.add_to(m)

# Show the map
m

In [ ]:
#Creating an instance of the datacube.
dc = datacube.Datacube(app='dc-visualize')

In the sample below we load the RGB bands of an extent covering the southern tip of Tasmanaia, Australia. Landsat images on AWS are referenced to WGS 84 so the extent bounds that are entered into the datacube.load() must match the coordinate system in which they're being stored. 

The datacube.load() command also reprojects into a coordinate system of choice as in the below example in which we've specified epsg - 28355 which corresponds to GDA94 Zone 55.

If you'd like to choose your own case-study area make sure you enter an appropriate epsg code.

If you'd like to visualize other bands add any of the below measurements,

Landsat 8 measurement options are:

             ('1', 'coastal_aerosol')
             ('2', 'blue')
             ('3', 'green')
             ('4', 'red')
             ('5', 'nir')
             ('6', 'swir1')
             ('7', 'swir2')
             ('8', 'panchromatic')
             ('9', 'cirrus')
             ('10', 'lwir1')
             ('11', 'lwir2')
             ('QUALITY', 'quality')]


In [ ]:
%%time
date_range = (datetime.datetime(2017,3,12),datetime.datetime(2017,3,14))
ds = dc.load(product='ls8_level1_usgs', x=(extent[0],extent[1]), y=(extent[2],extent[3]), 
             output_crs = 'epsg:28355', resolution = (-30,30), time = date_range, 
             measurements = ('red','green','blue'))

The datacube.load() returns an [xarray](https://xarray.pydata.org/en/stable/), to understand how to query a datacube it is important to understand the xarray. Click on the documention to find out more, otherwise a number of examples are provided below.

The xarray for landsat contains the dimensions of X,Y and Time - for each combination of these dimensions data listed as "Data Variables" can be retrieved. 

In [ ]:
print (ds) #viewing the data

In [ ]:
#Query xarray using index, as any python array 0 will retrieve the first record of each dimension
print (ds.isel(time = [0], x = [0], y=[0])) 


In [ ]:
#Extract all y measurements between x = 443200 and x = 443500
print (ds.sel(x = [443200,443500],  method='nearest')) 


Next we'll display the RGB bands that we have extracted from our cube for the first time slice (time = 0), in order to visualise a comprehendible RGB image we apply a histogram equalisation on each band before displaying.

In [ ]:
%pylab notebook
def callback(event):
    global x, y
    x, y = int(event.xdata + 0.5), int(event.ydata + 0.5)

figsize = [10,10]
time = 0
img_toshow = threeBandImage(ds,bands = ['red', 'green', 'blue'], time = time)
fig = plt.figure(figsize = figsize)
fig.canvas.mpl_connect('button_press_event', callback)

imshow(img_toshow)
ax = plt.gca()
ax.set_title(str(ds.time[time].values), fontweight = 'bold', fontsize = 16)
ax.set_xticklabels(ds.x.values)
ax.set_yticklabels(ds.y.values)
ax.set_xlabel('Easting', fontweight = 'bold')
ax.set_ylabel('Northing', fontweight = 'bold')

Next we'll select a single pixel from the above image and drill through the time dimension to calculate the change in nDVI and quality of the measurements.

To select a pixel simply click on the image above and run the below cell.

In [ ]:
pixelDrillExtent = []
for i in [[x,y],[x+1,y+1]]:
    xI,yI = i
    getLong = ds.isel(x=[xI], y=[yI]).isel(time =0).x.values
    getLat = ds.isel(x=[xI], y=[yI]).isel(time =0).y.values
    
    pixelDrillExtent.append(transformToWGS(getLong, getLat,28355))
print pixelDrillExtent

By expanding out the time extent and limiting the x,y extent in the datacube.load() we are able to retrieve cross section through time.


In [ ]:
%%time
date_range = (datetime.datetime(2004,3,12),datetime.datetime(2018,3,14))
ds1 = dc.load(product='ls8_level1_usgs', x=(pixelDrillExtent[0][0],pixelDrillExtent[1][0]), y=(pixelDrillExtent[0][1],pixelDrillExtent[1][1]), 
             output_crs = 'epsg:28355', resolution = (-30,30), time = date_range,measurements = ('red','nir','quality'))



In [ ]:
print (ds1)

In [ ]:
pixeldrill = ds1.isel(x = [0], y = [0])

ndvi = (pixeldrill.nir.values-pixeldrill.red.values)/(pixeldrill.nir.values+pixeldrill.red.values)

ds1['ndvi'] = (('time', 'y','x'), ndvi)
#print (ds1)

In [ ]:
print (np.ravel(np.ravel(ds1.time.values)))

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt

plt.plot(np.ravel(ds1.time.values),np.ravel(ndvi))
plt.axis = 'NDVI'
plt.show()